# Aircraft Crashes Data Collection And Cleaning

## Overview

Use web scraping to gather the data of all accidents since 1918.

## Data Collection

In [2]:
from bs4 import BeautifulSoup
import math
import pandas as pd
import re
import requests
from urllib.parse import unquote

In [ ]:
# Fetch total number of accidents
root_url = 'https://www.baaa-acro.com'

response = requests.get(root_url)

if response.status_code != 200:
	print('Failed to fetch page.')
else:
	soup = BeautifulSoup(response.content, 'html.parser')
	accident_files = soup.find('div', {'class': 'total-accident-files'})
	nb_crashes = int(accident_files.text.replace(',', ''))

In [ ]:
# Fetch details of every accident
nb_rows_per_page = 20
nb_pages = math.ceil(nb_crashes / nb_rows_per_page)
csv_path = 'data/crashes_scraped_data.csv'

for i in range(nb_pages):
	listing_url = '{}/crash-archives?page={}'.format(root_url, i)
	response = requests.get(listing_url)
	soup = BeautifulSoup(response.content, 'html.parser')
	anchors = soup.find_all('a', {'class': 'red-btn'})

	crash_list = []
	for j, a in enumerate(anchors):
		link = a['href']
		#print('Page {}, link {}: {}{}'.format(i, j + 1, root_url, link))
		details_url = root_url + link
		response = requests.get(details_url)
		soup = BeautifulSoup(response.content, 'html.parser')
		details = {}
		
		details_div = soup.find('div', {'class': 'crash-details'})
		
		date_div = details_div.find('div', {'class': 'crash-date'})
		details['date'] = date_div.find('span').next_sibling.text.strip() if date_div else None
		
		aircraft_div = details_div.find('div', {'class': 'crash-aircraft'})
		details['aircraft'] = aircraft_div.find('a').find('div').text if aircraft_div else None
		
		operator_div = details_div.find('div', {'class': 'crash-operator'})

		if operator_div:
			if (operator_div.find('img')): # Extract operator name from image link
				pattern = re.compile(r'(?<=target_id=).*(?= \(\d+\))')
				img_link = unquote(operator_div.find('img').parent['href'])
				details['operator'] = pattern.search(img_link).group(0)
			else:
				details['operator'] = operator_div.find('a').find('div').text
		else:
			details['operator'] = None

		reg_div = details_div.find('div', {'class': 'crash-registration'})
		details['registration'] = reg_div.find('div').text if reg_div else None
		
		flight_phase_div = details_div.find('div', {'class': 'crash-flight-phase'})
		details['flight_phase'] = flight_phase_div.find('a').find('div').text if flight_phase_div else None
		
		flight_type_div = details_div.find('div', {'class': 'crash-flight-type'})
		details['flight_type'] = flight_type_div.find('a').find('div').text if flight_type_div else None
		
		survivors_div = details_div.find('div', {'class': 'crash-survivors'})
		details['survivors'] = survivors_div.find('a').find('div').text if survivors_div else None
		
		site_div = details_div.find('div', {'class': 'crash-site'})
		details['site'] = site_div.find('a').find('div').text if site_div else None
		
		schedule_div = details_div.find('div', {'class': 'crash-schedule'})
		details['schedule'] = schedule_div.find('div').text if schedule_div else None
		
		msn_div = details_div.find('div', {'class': 'crash-construction-num'})
		details['msn'] = msn_div.find('div').text if msn_div else None
		
		yom_div = details_div.find('div', {'class': 'crash-yom'})
		details['yom'] = yom_div.find('div').text if yom_div else None

		flight_number = details_div.find('div', {'class': 'crash-flight-number'})
		details['flight_number'] = flight_number.find('div').text if flight_number else None
		
		location_div = details_div.find('div', {'class': 'crash-location'})
		if location_div:
			location_details = location_div.select('a')
			details['location'] = ', '.join(item.text.strip() for item in location_details) if location_details else None
		else:
			details['location'] = None
		
		country_div = details_div.find('div', {'class': 'crash-country'})
		details['country'] = country_div.find('a').find('div').text if country_div else None
		
		region_div = details_div.find('div', {'class': 'crash-region'})
		details['region'] = region_div.find('a').find('div').text if region_div else None
		
		crew_on_board_div = details_div.find('div', {'class': 'crash-crew-on-board'})
		details['crew_on_board'] = crew_on_board_div.find('div').text if crew_on_board_div else None
		
		crew_fatalities_div = details_div.find('div', {'class': 'crash-crew-fatalities'})
		details['crew_fatalities'] = crew_fatalities_div.find('div').text if crew_fatalities_div else None
		
		pax_on_board_div = details_div.find('div', {'class': 'crash-pax-on-board'})
		details['pax_on_board'] = pax_on_board_div.find('div').text if pax_on_board_div else None
		
		pax_fatalities_div = details_div.find('div', {'class': 'crash-pax-fatalities'})
		details['pax_fatalities'] = pax_fatalities_div.find('div').text if pax_fatalities_div else None
		
		others_div = details_div.find('div', {'class': 'crash-other-fatalities'})
		details['other_fatalities'] = others_div.find('div').text if others_div else None
		
		total_fatalities_div = details_div.find('div', {'class': 'crash-total-fatalities'})
		details['total_fatalities'] = total_fatalities_div.find('div').text if total_fatalities_div else None

		captain_hours_div = details_div.find('div', {'class': 'captain-total-flying-hours'})
		details['captain_flying_hours'] = captain_hours_div.find('div').text if captain_hours_div else None

		captain_hours_type_div = details_div.find('div', {'class': 'captain-total-hours-type'})
		details['captain_flying_hours_on_type'] = captain_hours_type_div.find('div').text if captain_hours_type_div else None

		copilot_hours_div = details_div.find('div', {'class': 'copilot-total-flying-hours'})
		details['copilot_flying_hours'] = copilot_hours_div.find('div').text if copilot_hours_div else None

		copilot_hours_type_div = details_div.find('div', {'class': 'copilot-total-hours-type'})
		details['copilot_flying_hours_on_type'] = copilot_hours_type_div.find('div').text if copilot_hours_type_div else None

		aircraft_hours_div = details_div.find('div', {'class': 'crash-aircraft-flight-hours'})
		details['aircraft_flying_hours'] = aircraft_hours_div.find('div').text if aircraft_hours_div else None

		aircraft_cycles_div = details_div.find('div', {'class': 'crash-aircraft-flight-cycles'})
		details['aircraft_flight_cycles'] = aircraft_cycles_div.find('div').text if aircraft_cycles_div else None
		
		crash_list.append(details)
	
	crashes_df = pd.DataFrame(crash_list)

	if i == 0:
		crashes_df.to_csv(csv_path, index=False)
	else:
		crashes_df.to_csv(csv_path, index=False, header=False, mode='a')


---

## Data Cleaning

In [19]:
# Load CSV
df = pd.read_csv('data/crashes_scraped_data.csv')
df.head()

,date,aircraft,operator,registration,flight_phase,flight_type,survivors,site,schedule,msn,...,pax_on_board,pax_fatalities,other_fatalities,total_fatalities,captain_flying_hours,captain_flying_hours_on_type,copilot_flying_hours,copilot_flying_hours_on_type,aircraft_flying_hours,aircraft_flight_cycles
0,"Mar 13, 2025 at 0733 LT",Cessna 525 CitationJet CJ2,LBL 525 CZ LLC,N525CZ,Takeoff (climb),Private,No,"Plain, Valley",Mesquite - Addison,525A-0380,...,0.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,"Mar 7, 2025",Antonov AN-32,Indian Air Force - Bharatiya Vayu Sena,NaN,Landing (descent or approach),Military,Yes,Airport (less than 10 km from airport),NaN,NaN,...,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,"Mar 4, 2025 at 0954 LT",BAe Jetstream 31,SAETA Perú (Servicios Aéreos Tarapota),OB-2178,Landing (descent or approach),Scheduled Revenue Flight,Yes,Airport (less than 10 km from airport),Iquitos - Güeppí,861,...,11.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,"Feb 25, 2025",Antonov AN-26,Sudanese Air Force - Al Quwwat al-Jawwiya As-S...,NaN,Takeoff (climb),Military,No,City,NaN,NaN,...,13.0,13.0,29.0,46,NaN,NaN,NaN,NaN,NaN,NaN
4,"Feb 23, 2025",Ilyushin II-76,Sudanese Air Force - Al Quwwat al-Jawwiya As-S...,1106,Flight,Military,No,Desert,NaN,10234 08265,...,0.0,0.0,0.0,7,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36080 entries, 0 to 36079
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          36080 non-null  object 
 1   aircraft                      36080 non-null  object 
 2   operator                      36078 non-null  object 
 3   registration                  34893 non-null  object 
 4   flight_phase                  35469 non-null  object 
 5   flight_type                   36023 non-null  object 
 6   survivors                     34804 non-null  object 
 7   site                          35714 non-null  object 
 8   schedule                      25707 non-null  object 
 9   msn                           28064 non-null  object 
 10  yom                           26335 non-null  float64
 11  flight_number                 2895 non-null   object 
 12  location                      36069 non-null  object 
 13  c

In [ ]:
# Add column for airplane age (year accident - yom)

## End